In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

from dvc.api import make_checkpoint

# Display full outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/project/spark-3.2.1-bin-hadoop3.2"

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("PySpark App") \
    .config("spark.jars", "postgresql-42.3.2.jar") \
    .getOrCreate()

### Data Transformation

In [19]:
# Read film data
df_film = spark.read.parquet("/project/Individual/parquet_files/origin.parquet").toPandas()
make_checkpoint()
df_film

,place,title,year,director,cast,rating,genre,duration,domestic_k,international_k,worldwide_k,distributor
0,187,Harry Potter and the Deathly Hallows: Part 2,2011,David Yates,"Daniel Radcliffe, Emma Watson",8.0748,Adventure,2 hr 10 min,381447.587,960912.354,1342359.942,Warner Bros.
1,188,Mr Smith Goes to Washington,1939,Frank Capra,"James Stewart, Jean Arthur",8.0725,Comedy,2 hr 9 min,144.738,144.738,1900.000,Columbia Pictures
2,189,The Grand Budapest Hotel,2014,Wes Anderson,"Ralph Fiennes, F. Murray Abraham",8.0716,Adventure,1 hr 39 min,59301.324,113780.865,173082.189,Fox Searchlight Pictures
3,190,Sherlock Jr,1924,Buster Keaton,"Buster Keaton, Kathryn McGuire",8.0704,Action,45 min,0.000,0.000,0.000,0
4,191,Le salaire de la peur,1953,Henri-Georges Clouzot,"Yves Montand, Charles Vanel",8.0702,Adventure,2 hr 11 min,0.000,0.000,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
245,58,The Lives of Others,2006,Florian Henckel von Donnersmarck,"Ulrich Mühe, Martina Gedeck",8.3869,Drama,2 hr 17 min,11286.112,66070.830,77356.942,Sony Pictures Classics
246,59,Sunset Blvd,1950,Billy Wilder,"William Holden, Gloria Swanson",8.3838,Drama,1 hr 50 min,299.645,0.428,300.073,Paramount Pictures
247,60,Paths of Glory,1957,Stanley Kubrick,"Kirk Douglas, Ralph Meeker",8.3712,Drama,1 hr 28 min,0.000,0.000,0.000,United Artists
248,61,The Shining,1980,Stanley Kubrick,"Jack Nicholson, Shelley Duvall",8.3683,Drama,2 hr 26 min,45634.352,1665.108,47299.460,Warner Bros.


In [20]:
# A glance at the duration column
df_film['duration']

0      2 hr 10 min
1       2 hr 9 min
2      1 hr 39 min
3           45 min
4      2 hr 11 min
          ...     
245    2 hr 17 min
246    1 hr 50 min
247    1 hr 28 min
248    2 hr 26 min
249     2 hr 5 min
Name: duration, Length: 250, dtype: object

In [21]:
# Change the format of unique running times
# Such as less than 1 hour (no 'hr') or exactly whole hours (no 'min')
less_1hr = df_film[df_film['duration'].str.contains('hr')== False]
less_1hr

,place,title,year,director,cast,rating,genre,duration,domestic_k,international_k,worldwide_k,distributor
3,190,Sherlock Jr,1924,Buster Keaton,"Buster Keaton, Kathryn McGuire",8.0704,Action,45 min,0.0,0.0,0.0,0


In [22]:
whole_hrs = df_film[df_film['duration'].str.contains('min')== False]
whole_hrs

,place,title,year,director,cast,rating,genre,duration,domestic_k,international_k,worldwide_k,distributor
16,203,Mad Max: Fury Road,2015,George Miller,"Tom Hardy, Charlize Theron",8.0611,Action,2 hr,154109.060,221600.410,375709.470,Warner Bros.
24,211,Platoon,1986,Oliver Stone,"Charlie Sheen, Tom Berenger",8.0478,Drama,2 hr,138530.565,15.067,138545.632,Orion Pictures
25,212,Rocky,1976,John G. Avildsen,"Sylvester Stallone, Talia Shire",8.0471,Drama,2 hr,117235.147,15.255,117250.402,United Artists
72,71,Oldeuboi,2003,Park Chan-wook,"Choi Min-sik, Yoo Ji-Tae",8.3201,Action,2 hr,707.481,14487.112,15194.593,Tartan
134,133,The Wolf of Wall Street,2013,Martin Scorsese,"Leonardo DiCaprio, Jonah Hill",8.1702,Biography,3 hr,116900.694,275100.000,392000.694,Paramount Pictures


In [23]:
# Mannually fix the format
df_film.at[3, 'duration'] = '0 hr 45 min'
df_film.at[16, 'duration'] = '2 hr 0 min'
df_film.at[24, 'duration'] = '2 hr 0 min'
df_film.at[25, 'duration'] = '2 hr 0 min'
df_film.at[72, 'duration'] = '2 hr 0 min'
df_film.at[134, 'duration'] = '3 hr 0 min'
make_checkpoint()

In [24]:
# Clean the duration column for transformation
df_film['duration'] = df_film['duration'].str.replace(' ', '')
df_film['duration'] = df_film['duration'].str.replace('min', '')
df_film['duration'] = df_film['duration'].str.replace('hr', ':')
make_checkpoint()

In [25]:
df_film['duration']

0      2:10
1       2:9
2      1:39
3      0:45
4      2:11
       ... 
245    2:17
246    1:50
247    1:28
248    2:26
249     2:5
Name: duration, Length: 250, dtype: object

In [26]:
# Convert duration into minutes
df_film['duration'] = df_film['duration'].str.split(':').apply(lambda x: int(x[0]) * 60 + int(x[1]))
make_checkpoint()

In [27]:
# Create duration rank column
df_film['duration_rank'] = df_film['duration'].rank(ascending=False)
make_checkpoint()

In [28]:
# Create duration by genre column
df_film['duration_rank_bygenre'] = df_film.groupby('genre')['duration'].rank(ascending=False)
make_checkpoint()

In [29]:
# Calculate the percentage of domestic and international box office income
df_film['dom_pct'] = df_film['domestic_k']/df_film['worldwide_k']
df_film['int_pct'] = df_film['international_k']/df_film['worldwide_k']
df_film['dom_pct'] = df_film['dom_pct'].round(decimals = 4)
df_film['int_pct'] = df_film['int_pct'].round(decimals = 4)
make_checkpoint()

In [30]:
# Check the realistic feasibility
df_film['check'] = df_film['dom_pct'] + df_film['int_pct']
df_film['check'] = df_film['check'].fillna(0)
make_checkpoint()

# Find the rows that do not make sense in real life
df_film.loc[(df_film['check'] != 1.0000) & (df_film['check'] != 0)]

,place,title,year,director,cast,rating,genre,duration,domestic_k,international_k,worldwide_k,distributor,duration_rank,duration_rank_bygenre,dom_pct,int_pct,check
1,188,Mr Smith Goes to Washington,1939,Frank Capra,"James Stewart, Jean Arthur",8.0725,Comedy,129,144.738,144.738,1900.000,Columbia Pictures,115.0,3.5,0.0762,0.0762,0.1524
18,205,La passion de Jeanne d'Arc,1928,Carl Theodor Dreyer,"Maria Falconetti, Eugene Silvain",8.0578,Biography,110,21.877,21.877,6.408,Janus Films,182.5,22.0,3.4140,3.4140,6.8280
27,214,Stand by Me,1986,Rob Reiner,"Wil Wheaton, River Phoenix",8.0429,Adventure,89,52287.414,52287.414,242.795,Columbia Pictures,235.5,35.0,215.3562,215.3562,430.7124
37,224,The Exorcist,1973,William Friedkin,"Ellen Burstyn, Max von Sydow",8.0289,Horror,122,233005.644,136017.945,441306.145,Warner Bros.,139.5,1.0,0.5280,0.3082,0.8362
58,245,Gandhi,1982,Richard Attenborough,"Ben Kingsley, John Gielgud",8.0078,Biography,191,52767.889,52767.889,131.153,Columbia Pictures,9.0,2.0,402.3384,402.3384,804.6768
61,248,Beauty and the Beast,1991,Gary Trousdale,"Paige O'Hara, Robby Benson",8.0045,Animation,84,218967.620,186043.788,424967.620,Walt Disney Studios Motion Pictures,245.0,5.0,0.5153,0.4378,0.9531
76,75,Das Boot,1981,Wolfgang Petersen,"Jürgen Prochnow, Herbert Grönemeyer",8.3124,Drama,149,11487.676,11487.676,26.994,Columbia Pictures,55.5,12.5,425.5641,425.5641,851.1282
87,86,Star Wars: Episode VI - Return of the Jedi,1983,Richard Marquand,"Mark Hamill, Harrison Ford",8.2687,Action,131,309306.177,122009.457,475106.177,Twentieth Century Fox,101.5,28.0,0.6510,0.2568,0.9078
104,103,Full Metal Jacket,1987,Stanley Kubrick,"Matthew Modine, R. Lee Ermey",8.2408,Drama,116,46357.676,46357.676,2217.307,Warner Bros.,167.0,49.5,20.9072,20.9072,41.8144
160,159,"Lock, Stock and Two Smoking Barrels",1998,Guy Ritchie,"Jason Flemyng, Dexter Fletcher",8.1244,Action,107,3753.929,3753.929,143.321,Gramercy Pictures (I),190.5,44.5,26.1925,26.1925,52.3850


In [31]:
# Mannually fix the films that do not have international incomes
df_film.at[1, 'international_k'] = 0.000
df_film.at[1, 'worldwide_k'] = 144.738
df_film.at[18, 'international_k'] = 0.000
df_film.at[18, 'worldwide_k'] = 21.877
df_film.at[27, 'international_k'] = 0.000
df_film.at[27, 'worldwide_k'] = 52287.414
df_film.at[58, 'international_k'] = 0.000
df_film.at[58, 'worldwide_k'] = 52767.889
df_film.at[76, 'international_k'] = 0.000
df_film.at[76, 'worldwide_k'] = 11487.676
df_film.at[104, 'international_k'] = 0.000
df_film.at[104, 'worldwide_k'] = 46357.676
df_film.at[160, 'international_k'] = 0.000
df_film.at[160, 'worldwide_k'] = 3753.929
df_film.at[164, 'international_k'] = 0.000
df_film.at[164, 'worldwide_k'] = 27200.000
df_film.at[179, 'international_k'] = 0.000
df_film.at[179, 'worldwide_k'] = 933.933
df_film.at[226, 'international_k'] = 0.000
df_film.at[226, 'worldwide_k'] = 23341.568
df_film.at[235, 'international_k'] = 0.000
df_film.at[235, 'worldwide_k'] = 516.962
make_checkpoint()

In [32]:
# Find other missing box office incomes data
df_film.loc[(df_film['domestic_k'] == 0) & (df_film['international_k'] == 0) & (df_film['worldwide_k'] == 0)]

,place,title,year,director,cast,rating,genre,duration,domestic_k,international_k,worldwide_k,distributor,duration_rank,duration_rank_bygenre,dom_pct,int_pct,check
3,190,Sherlock Jr,1924,Buster Keaton,"Buster Keaton, Kathryn McGuire",8.0704,Action,45,0.0,0.0,0.0,0,250.0,48.0,NaN,NaN,0.0
4,191,Le salaire de la peur,1953,Henri-Georges Clouzot,"Yves Montand, Charles Vanel",8.0702,Adventure,131,0.0,0.0,0.0,0,101.5,14.0,NaN,NaN,0.0
6,193,Barry Lyndon,1975,Stanley Kubrick,"Ryan O'Neal, Marisa Berenson",8.0686,Adventure,185,0.0,0.0,0.0,0,11.0,3.0,NaN,NaN,0.0
8,195,Det sjunde inseglet,1957,Ingmar Bergman,"Max von Sydow, Gunnar Björnstrand",8.0674,Drama,96,0.0,0.0,0.0,Janus Films,220.5,61.0,NaN,NaN,0.0
12,199,Klaus,2019,Sergio Pablos,"Jason Schwartzman, J.K. Simmons",8.0652,Adventure,96,0.0,0.0,0.0,0,220.5,28.5,NaN,NaN,0.0
17,204,Mary and Max,2009,Adam Elliot,"Toni Collette, Philip Seymour Hoffman",8.0604,Animation,92,0.0,0.0,0.0,0,229.5,2.0,NaN,NaN,0.0
22,209,Tôkyô monogatari,1953,Yasujirô Ozu,"Chishû Ryû, Chieko Higashiyama",8.0522,Drama,136,0.0,0.0,0.0,0,86.5,25.0,NaN,NaN,0.0
42,229,Rebecca,1940,Alfred Hitchcock,"Laurence Olivier, Joan Fontaine",8.0234,Drama,130,0.0,0.0,0.0,United Artists,107.5,31.0,NaN,NaN,0.0
43,230,To Be or Not to Be,1942,Ernst Lubitsch,"Carole Lombard, Jack Benny",8.0234,Comedy,99,0.0,0.0,0.0,United Artists,211.5,19.0,NaN,NaN,0.0
44,231,The Grapes of Wrath,1940,John Ford,"Henry Fonda, Jane Darwell",8.0225,Drama,129,0.0,0.0,0.0,Twentieth Century Fox,115.0,34.5,NaN,NaN,0.0


In [33]:
# Fill in missing values mannully
df_film.at[4, 'international_k'] = 1.098
df_film.at[4, 'worldwide_k'] = 1.098
df_film.at[6, 'international_k'] = 198.992
df_film.at[6, 'worldwide_k'] = 198.992
df_film.at[8, 'international_k'] = 286.085
df_film.at[8, 'worldwide_k'] = 286.085
df_film.at[17, 'international_k'] = 1740.429
df_film.at[17, 'worldwide_k'] = 1740.429
df_film.at[22, 'international_k'] = 40.468
df_film.at[22, 'worldwide_k'] = 40.468
df_film.at[42, 'international_k'] = 72.275
df_film.at[42, 'worldwide_k'] = 72.275
df_film.at[46, 'international_k'] = 46749.646
df_film.at[46, 'worldwide_k'] = 46749.646
df_film.at[48, 'international_k'] = 14.480
df_film.at[48, 'worldwide_k'] = 14.480
df_film.at[49, 'international_k'] = 18612.999
df_film.at[49, 'worldwide_k'] = 18612.999
df_film.at[51, 'international_k'] = 90.556
df_film.at[51, 'worldwide_k'] = 90.556
df_film.at[64, 'international_k'] = 7.693
df_film.at[64, 'worldwide_k'] = 7.693
df_film.at[93, 'domestic_k'] = 46.808
df_film.at[93, 'worldwide_k'] = 46.808
df_film.at[97, 'domestic_k'] = 35.566
df_film.at[97, 'worldwide_k'] = 35.566
df_film.at[105, 'international_k'] = 14.190
df_film.at[105, 'worldwide_k'] = 14.190
df_film.at[110, 'domestic_k'] = 156000.000
df_film.at[110, 'worldwide_k'] = 156000.000
df_film.at[126, 'domestic_k'] = 15000.000
df_film.at[126, 'worldwide_k'] = 15000.000
df_film.at[129, 'international_k'] = 41.960
df_film.at[129, 'worldwide_k'] = 41.960
df_film.at[130, 'international_k'] = 195.088
df_film.at[130, 'worldwide_k'] = 195.088
df_film.at[135, 'international_k'] = 12.180
df_film.at[135, 'worldwide_k'] = 12.180
df_film.at[146, 'domestic_k'] = 5014.000
df_film.at[146, 'worldwide_k'] = 5014.000
df_film.at[147, 'domestic_k'] = 46.808
df_film.at[147, 'worldwide_k'] = 46.808
df_film.at[148, 'international_k'] = 228.178
df_film.at[148, 'worldwide_k'] = 228.178
df_film.at[174, 'international_k'] = 26.916
df_film.at[174, 'worldwide_k'] = 26.916
df_film.at[183, 'international_k'] = 14.524
df_film.at[183, 'worldwide_k'] = 14.524
df_film.at[192, 'international_k'] = 0.955
df_film.at[192, 'worldwide_k'] = 0.955
df_film.at[234, 'international_k'] = 15.222
df_film.at[234, 'worldwide_k'] = 15.222
df_film.at[247, 'international_k'] = 5.252
df_film.at[247, 'worldwide_k'] = 5.252
df_film.at[249, 'international_k'] = 970.214
df_film.at[249, 'worldwide_k'] = 970.214
make_checkpoint()

In [34]:
# Check again
df_film['dom_pct'] = df_film['domestic_k']/df_film['worldwide_k']
df_film['int_pct'] = df_film['international_k']/df_film['worldwide_k']
df_film['dom_pct'] = df_film['dom_pct'].round(decimals = 4)
df_film['int_pct'] = df_film['int_pct'].round(decimals = 4)
df_film['check'] = df_film['dom_pct'] + df_film['int_pct']
df_film['check'] = df_film['check'].fillna(0)
make_checkpoint()
df_film['check'].value_counts()

1.0000    234
0.0000     10
0.8980      1
0.8470      1
0.8362      1
0.9531      1
0.9945      1
0.9078      1
Name: check, dtype: int64

In [35]:
# Drop the check column
df_film = df_film.drop('check', axis = 1)
make_checkpoint()

In [36]:
# Create place by genre column
df_film['place_bygenre'] = df_film.groupby('genre')['place'].rank(ascending=False)
make_checkpoint()

In [37]:
# Create income rank column
df_film['income_rank'] = df_film['worldwide_k'].rank(ascending=False)
make_checkpoint()

In [38]:
# Create income rank by genre column
df_film['income_rank_bygenre'] = df_film.groupby('genre')['worldwide_k'].rank(ascending=False)
make_checkpoint()

In [39]:
# Calculate the difference between rating rank (place) and income rank
df_film['rank_diff'] = df_film['income_rank'] - df_film['place']
make_checkpoint()

In [40]:
# Check for missing values
df_film.isnull().sum()

place                     0
title                     0
year                      0
director                  0
cast                      0
rating                    0
genre                     0
duration                  0
domestic_k                0
international_k           0
worldwide_k               0
distributor               0
duration_rank             0
duration_rank_bygenre     0
dom_pct                  10
int_pct                  10
place_bygenre             0
income_rank               0
income_rank_bygenre       0
rank_diff                 0
dtype: int64

In [41]:
# Fill in NaN with 0
df_film.fillna(0, inplace=True)
make_checkpoint()

In [42]:
# Check again
df_film.isnull().sum()

place                    0
title                    0
year                     0
director                 0
cast                     0
rating                   0
genre                    0
duration                 0
domestic_k               0
international_k          0
worldwide_k              0
distributor              0
duration_rank            0
duration_rank_bygenre    0
dom_pct                  0
int_pct                  0
place_bygenre            0
income_rank              0
income_rank_bygenre      0
rank_diff                0
dtype: int64

In [43]:
# Adjust the order of the columns
df_film = df_film[['title', 'year', 'director', 'cast', 'distributor', 'genre', 'rating', 'place', 'place_bygenre', 'duration', 'duration_rank', 'duration_rank_bygenre', 'domestic_k', 'international_k', 'worldwide_k', 'dom_pct', 'int_pct', 'income_rank', 'income_rank_bygenre', 'rank_diff']]
make_checkpoint()

In [44]:
# Final version of film data frame
df_film

,title,year,director,cast,distributor,genre,rating,place,place_bygenre,duration,duration_rank,duration_rank_bygenre,domestic_k,international_k,worldwide_k,dom_pct,int_pct,income_rank,income_rank_bygenre,rank_diff
0,Harry Potter and the Deathly Hallows: Part 2,2011,David Yates,"Daniel Radcliffe, Emma Watson",Warner Bros.,Adventure,8.0748,187,14.0,130,107.5,15.0,381447.587,960912.354,1342359.942,0.2842,0.7158,4.0,1.0,-183.0
1,Mr Smith Goes to Washington,1939,Frank Capra,"James Stewart, Jean Arthur",Columbia Pictures,Comedy,8.0725,188,8.0,129,115.0,3.5,144.738,0.000,144.738,1.0000,0.0000,216.0,20.0,28.0
2,The Grand Budapest Hotel,2014,Wes Anderson,"Ralph Fiennes, F. Murray Abraham",Fox Searchlight Pictures,Adventure,8.0716,189,13.0,99,211.5,26.0,59301.324,113780.865,173082.189,0.3426,0.6574,87.0,20.0,-102.0
3,Sherlock Jr,1924,Buster Keaton,"Buster Keaton, Kathryn McGuire",0,Action,8.0704,190,10.0,45,250.0,48.0,0.000,0.000,0.000,0.0000,0.0000,245.5,47.0,55.5
4,Le salaire de la peur,1953,Henri-Georges Clouzot,"Yves Montand, Charles Vanel",0,Adventure,8.0702,191,12.0,131,101.5,14.0,0.000,1.098,1.098,0.0000,1.0000,239.0,36.0,48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,The Lives of Others,2006,Florian Henckel von Donnersmarck,"Ulrich Mühe, Martina Gedeck",Sony Pictures Classics,Drama,8.3869,58,55.0,137,82.5,24.0,11286.112,66070.830,77356.942,0.1459,0.8541,114.0,20.0,56.0
246,Sunset Blvd,1950,Billy Wilder,"William Holden, Gloria Swanson",Paramount Pictures,Drama,8.3838,59,54.0,110,182.5,53.0,299.645,0.428,300.073,0.9986,0.0014,210.0,57.0,151.0
247,Paths of Glory,1957,Stanley Kubrick,"Kirk Douglas, Ralph Meeker",United Artists,Drama,8.3712,60,53.0,88,239.0,66.0,0.000,5.252,5.252,0.0000,1.0000,238.0,67.0,178.0
248,The Shining,1980,Stanley Kubrick,"Jack Nicholson, Shelley Duvall",Warner Bros.,Drama,8.3683,61,52.0,146,62.5,15.5,45634.352,1665.108,47299.460,0.9648,0.0352,132.0,26.0,71.0


In [45]:
# Convert the data frame into spark data frame
df_film_spark = spark.createDataFrame(df_film)

In [46]:
df_film_spark.printSchema()

root
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- distributor: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- place: long (nullable = true)
 |-- place_bygenre: double (nullable = true)
 |-- duration: long (nullable = true)
 |-- duration_rank: double (nullable = true)
 |-- duration_rank_bygenre: double (nullable = true)
 |-- domestic_k: double (nullable = true)
 |-- international_k: double (nullable = true)
 |-- worldwide_k: double (nullable = true)
 |-- dom_pct: double (nullable = true)
 |-- int_pct: double (nullable = true)
 |-- income_rank: double (nullable = true)
 |-- income_rank_bygenre: double (nullable = true)
 |-- rank_diff: double (nullable = true)



In [47]:
# Convert the data frame into parquet format
df_film_spark.write.parquet("/project/Individual/parquet_files/film.parquet", mode = 'overwrite')
make_checkpoint()